# Logistic Regression Implementation from scratch

In [1]:
#Getting all the important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

### Exploring our dataset

In [2]:
cardio = pd.read_csv("cardio_train.csv", sep = ';')

We have a total of 11 input columns and 1 output column which represents whether the person has a Cardiovascular disease or not.
We have roughly 35000 entries for each classification so it is a balanced dataset. No more processing is required on it.

In [3]:
cardio.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
sample_0 = cardio[cardio["cardio"] == 0][:11000]
sample_1 = cardio[cardio["cardio"] == 1][:11000]

In [5]:
sample = pd.concat([sample_0,sample_1])

In [6]:
sample = sample.sample(frac=1).reset_index(drop=True)
sample.head(20)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,14472,18796,1,165,69.0,140,90,2,1,0,0,1,0
1,9475,22588,2,170,65.0,120,80,1,1,0,0,1,0
2,20586,18489,1,157,61.0,120,80,1,1,0,0,1,0
3,15288,23476,1,171,120.0,150,100,2,2,0,0,0,1
4,15469,23492,2,171,73.0,120,80,1,1,0,0,0,0
5,983,19687,1,150,55.0,110,70,1,1,0,0,1,0
6,13608,17999,2,166,76.0,130,80,1,1,0,0,1,1
7,185,16889,1,156,60.0,120,80,1,1,0,0,1,0
8,15714,23253,1,155,67.0,120,80,3,2,0,0,1,1
9,8625,18240,1,160,50.0,110,80,1,1,0,0,1,0


In [7]:
#Slitting the input and output columns
x = sample[["age","gender","weight", "ap_hi","ap_lo","cholesterol","gluc","smoke","alco","active"]]
y = sample["cardio"]

In [8]:
x.head()

,age,gender,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,18796,1,69.0,140,90,2,1,0,0,1
1,22588,2,65.0,120,80,1,1,0,0,1
2,18489,1,61.0,120,80,1,1,0,0,1
3,23476,1,120.0,150,100,2,2,0,0,0
4,23492,2,73.0,120,80,1,1,0,0,0


In [9]:
#splitting dataset into test and train samoples
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

In [10]:
X = train_x.values
Y = train_y.values
X = np.float32(X)
Y = np.float32(Y) 
Y = Y.reshape(Y.shape[0],-1)

In [11]:
print(Y.shape)
print(X.shape)

(20000, 1)
(20000, 10)


Defining helper functions for the aglorithm

In [12]:
def get_sigmoid(inp):
    
    #simply returning sigmoid our values
    return (1/(1+np.exp(-inp))) - 0.00000001
    

In [13]:
def get_loss(y_hat, y, n):
    
    #simply returning the loss calculated using our loss function
    return (1/n)*(np.sum(-y*(np.log(y_hat)) - (1-y)*np.log(1-y_hat)))
    

In [14]:
def update_params(n, X, y_hat, y, alpha, w_old):
  
    #looping to update every parameter
    diff = y_hat-y
    update = (1/n)*np.dot(diff.T,X)
   
    #applying the update equation
    w_new = w_old-(alpha)*(update)
    
    #return the updated weights
    return w_new

In [15]:
def train_logistic_regression(X,y, alpha = 0.0001, epochs = 25000):
    
    #first we get the number of examples present in our dataset
    n = len(X)
    
    #normalise data using min_max scaling
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    
    #Randomly initialising weights of our model
    w = np.random.rand(1,X.shape[1])
    print("Original weights =",w)
    #setting an initial value for loss (a high value)
    loss = 1000
    for i in range(epochs):
        print("Loss ================>",loss)
        dot = np.dot(X, w.T)
        y_hat = get_sigmoid(dot)
        
        #calculate the loss against the predicted outputs
        loss = get_loss(y_hat, y, n)

        #updating the weights based on the loss
        w = update_params(n, X, y_hat, y, alpha, w)
    
    return w, y_hat

In [17]:
%%time
weights, out = train_logistic_regression(X,Y)

Original weights = [[0.17467056 0.50280197 0.67611796 0.48841772 0.40160956 0.51317784
  0.17093419 0.99168994 0.9059328  0.16534312]]
Loss ================> 1000
Loss ================> 0.7885543807019558
Loss ================> 0.7885489819854472
Loss ================> 0.7885435835534315
Loss ================> 0.7885381854058963
Loss ================> 0.7885327875428293
Loss ================> 0.7885273899642187
Loss ================> 0.7885219926700521
Loss ================> 0.7885165956603171
Loss ================> 0.7885111989350018
Loss ================> 0.788505802494094
Loss ================> 0.7885004063375814
Loss ================> 0.788495010465452
Loss ================> 0.7884896148776936
Loss ================> 0.7884842195742938
Loss ================> 0.7884788245552407
Loss ================> 0.7884734298205218
Loss ================> 0.7884680353701252
Loss ================> 0.7884626412040387
Loss ================> 0.7884572473222499
Loss ================> 0.7884518537247468

Loss ================> 0.7871922148968353
Loss ================> 0.7871868880505483
Loss ================> 0.7871815614856654
Loss ================> 0.7871762352021744
Loss ================> 0.7871709092000634
Loss ================> 0.7871655834793202
Loss ================> 0.7871602580399324
Loss ================> 0.7871549328818882
Loss ================> 0.7871496080051753
Loss ================> 0.7871442834097817
Loss ================> 0.7871389590956951
Loss ================> 0.7871336350629033
Loss ================> 0.7871283113113945
Loss ================> 0.7871229878411561
Loss ================> 0.7871176646521764
Loss ================> 0.7871123417444429
Loss ================> 0.7871070191179439
Loss ================> 0.7871016967726668
Loss ================> 0.7870963747085997
Loss ================> 0.7870910529257304
Loss ================> 0.7870857314240469
Loss ================> 0.7870804102035369
Loss ================> 0.7870750892641883
Loss ================> 0.787069768

Loss ================> 0.7858271836378385
Loss ================> 0.7858219289918905
Loss ================> 0.7858166746242206
Loss ================> 0.7858114205348168
Loss ================> 0.785806166723667
Loss ================> 0.7858009131907592
Loss ================> 0.7857956599360814
Loss ================> 0.7857904069596211
Loss ================> 0.7857851542613666
Loss ================> 0.7857799018413056
Loss ================> 0.7857746496994261
Loss ================> 0.7857693978357161
Loss ================> 0.7857641462501633
Loss ================> 0.7857588949427556
Loss ================> 0.7857536439134812
Loss ================> 0.7857483931623278
Loss ================> 0.7857431426892832
Loss ================> 0.7857378924943356
Loss ================> 0.7857326425774727
Loss ================> 0.7857273929386824
Loss ================> 0.7857221435779527
Loss ================> 0.7857168944952714
Loss ================> 0.7857116456906266
Loss ================> 0.7857063971

Loss ================> 0.7845014123157873
Loss ================> 0.7844962279644336
Loss ================> 0.7844910438882923
Loss ================> 0.7844858600873513
Loss ================> 0.7844806765615988
Loss ================> 0.7844754933110225
Loss ================> 0.7844703103356104
Loss ================> 0.7844651276353507
Loss ================> 0.784459945210231
Loss ================> 0.7844547630602394
Loss ================> 0.784449581185364
Loss ================> 0.7844443995855926
Loss ================> 0.7844392182609132
Loss ================> 0.7844340372113137
Loss ================> 0.7844288564367822
Loss ================> 0.7844236759373063
Loss ================> 0.7844184957128745
Loss ================> 0.7844133157634742
Loss ================> 0.7844081360890937
Loss ================> 0.7844029566897209
Loss ================> 0.7843977775653439
Loss ================> 0.7843925987159502
Loss ================> 0.7843874201415282
Loss ================> 0.78438224184

Loss ================> 0.7832343259127995
Loss ================> 0.7832292089011594
Loss ================> 0.7832240921617742
Loss ================> 0.7832189756946318
Loss ================> 0.7832138594997204
Loss ================> 0.7832087435770276
Loss ================> 0.7832036279265419
Loss ================> 0.783198512548251
Loss ================> 0.7831933974421427
Loss ================> 0.7831882826082054
Loss ================> 0.7831831680464271
Loss ================> 0.7831780537567955
Loss ================> 0.7831729397392988
Loss ================> 0.7831678259939249
Loss ================> 0.7831627125206619
Loss ================> 0.7831575993194978
Loss ================> 0.7831524863904206
Loss ================> 0.7831473737334181
Loss ================> 0.7831422613484788
Loss ================> 0.7831371492355901
Loss ================> 0.7831320373947405
Loss ================> 0.7831269258259175
Loss ================> 0.7831218145291097
Loss ================> 0.7831167035

...

Loss ================> 0.716787831417272
Loss ================> 0.7167864027768468
Loss ================> 0.7167849742155532
Loss ================> 0.7167835457333861
Loss ================> 0.7167821173303409
Loss ================> 0.7167806890064129
Loss ================> 0.716779260761597
Loss ================> 0.7167778325958888
Loss ================> 0.7167764045092833
Loss ================> 0.7167749765017756
Loss ================> 0.7167735485733611
Loss ================> 0.716772120724035
Loss ================> 0.7167706929537923
Loss ================> 0.7167692652626284
Loss ================> 0.7167678376505385
Loss ================> 0.7167664101175181
Loss ================> 0.7167649826635615
Loss ================> 0.716763555288665
Loss ================> 0.7167621279928231
Loss ================> 0.7167607007760314
Loss ================> 0.7167592736382848
Loss ================> 0.7167578465795788
Loss ================> 0.7167564195999084
Loss ================> 0.7167549926992

Loss ================> 0.7165093241909521
Loss ================> 0.7165079109690589
Loss ================> 0.7165064978253631
Loss ================> 0.71650508475986
Loss ================> 0.7165036717725448
Loss ================> 0.7165022588634127
Loss ================> 0.7165008460324592
Loss ================> 0.7164994332796795
Loss ================> 0.7164980206050686
Loss ================> 0.7164966080086221
Loss ================> 0.7164951954903349
Loss ================> 0.7164937830502027
Loss ================> 0.7164923706882205
Loss ================> 0.7164909584043835
Loss ================> 0.7164895461986872
Loss ================> 0.7164881340711267
Loss ================> 0.7164867220216973
Loss ================> 0.7164853100503943
Loss ================> 0.7164838981572128
Loss ================> 0.7164824863421482
Loss ================> 0.7164810746051957
Loss ================> 0.7164796629463508
Loss ================> 0.7164782513656085
Loss ================> 0.71647683986

Loss ================> 0.7161542592320301
Loss ================> 0.7161528656436947
Loss ================> 0.7161514721323662
Loss ================> 0.7161500786980395
Loss ================> 0.7161486853407102
Loss ================> 0.7161472920603732
Loss ================> 0.7161458988570245
Loss ================> 0.7161445057306589
Loss ================> 0.7161431126812718
Loss ================> 0.716141719708859
Loss ================> 0.7161403268134152
Loss ================> 0.716138933994936
Loss ================> 0.716137541253417
Loss ================> 0.7161361485888529
Loss ================> 0.7161347560012395
Loss ================> 0.7161333634905721
Loss ================> 0.7161319710568457
Loss ================> 0.7161305787000561
Loss ================> 0.7161291864201983
Loss ================> 0.7161277942172678
Loss ================> 0.7161264020912598
Loss ================> 0.7161250100421698
Loss ================> 0.716123618069993
Loss ================> 0.7161222261747

Loss ================> 0.7158151205413549
Loss ================> 0.7158137456805819
Loss ================> 0.7158123708956783
Loss ================> 0.7158109961866396
Loss ================> 0.7158096215534608
Loss ================> 0.7158082469961378
Loss ================> 0.7158068725146655
Loss ================> 0.7158054981090395
Loss ================> 0.7158041237792554
Loss ================> 0.7158027495253082
Loss ================> 0.7158013753471935
Loss ================> 0.7158000012449067
Loss ================> 0.7157986272184432
Loss ================> 0.7157972532677983
Loss ================> 0.7157958793929675
Loss ================> 0.7157945055939462
Loss ================> 0.7157931318707296
Loss ================> 0.7157917582233133
Loss ================> 0.7157903846516925
Loss ================> 0.7157890111558629
Loss ================> 0.7157876377358197
Loss ================> 0.7157862643915581
Loss ================> 0.715784891123074
Loss ================> 0.7157835179

Loss ================> 0.7154927473167217
Loss ================> 0.7154913902338279
Loss ================> 0.7154900332257225
Loss ================> 0.7154886762924011
Loss ================> 0.7154873194338587
Loss ================> 0.7154859626500913
Loss ================> 0.7154846059410941
Loss ================> 0.7154832493068626
Loss ================> 0.7154818927473922
Loss ================> 0.7154805362626783
Loss ================> 0.7154791798527165
Loss ================> 0.7154778235175021
Loss ================> 0.7154764672570307
Loss ================> 0.7154751110712979
Loss ================> 0.7154737549602987
Loss ================> 0.715472398924029
Loss ================> 0.7154710429624839
Loss ================> 0.715469687075659
Loss ================> 0.7154683312635498
Loss ================> 0.7154669755261517
Loss ================> 0.7154656198634602
Loss ================> 0.7154642642754707
Loss ================> 0.7154629087621787
Loss ================> 0.71546155332

In [18]:
weights.shape

(1, 10)

In [19]:
def predict_logistic_regression(weights, x_test):
    
    #doing the neccesary calculations before applying sigmoid funciton
    X = x_test.values
    #normalise data using min_max scaling
    scaler = MinMaxScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    product = np.dot(X,weights.T)
    y_preds = get_sigmoid(product)
    
    return [1 if i >= 0.5 else 0 for i in y_preds]

In [20]:
y_pred = predict_logistic_regression(weights, test_x)

In [21]:
def get_accuracy(y_pred, y):
    
    assert len(y_pred) == len(y)
    n = len(y)
    Y = y.values
    
    count = 0
    for i in range(n):
        if y_pred[i] == Y[i]:
            count+=1
     
    return print("Accuracy:", round(((count/n)*100),2), "%")

In [22]:
get_accuracy(y_pred, test_y)

Accuracy: 51.7 %


In [ ]:
get_accuracy(out, train_y)

In [ ]:
out

In [ ]:
train_y